In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Data Ingestion Techniques
loader - UnstructuredLoader

In [2]:
from langchain_community.document_loaders import UnstructuredURLLoader
url = ["https://www.paloaltonetworks.ca/cyberpedia/ai-governance"]
reader = UnstructuredURLLoader(urls=url)
doc = reader.load()
doc

[Document(metadata={'source': 'https://www.paloaltonetworks.ca/cyberpedia/ai-governance'}, page_content="Table of Contents\n\nUnderstanding AI Governance\n\nAI Governance Challenges\n\nEstablishing Ethical Guidelines\n\nNavigating Regulatory Frameworks\n\nAccountability Mechanisms\n\nEnsuring Transparency and Explainability\n\nImplementing AI Governance Frameworks\n\nMonitoring and Continuous Improvement\n\nSecuring AI Systems\n\nAI Governance FAQs\n\nCloud Security\n\nWhat Is AI Governance?\n\n3 min. read\n\nTable of Contents\n\nUnderstanding AI Governance\n\nAI Governance Challenges\n\nEstablishing Ethical Guidelines\n\nNavigating Regulatory Frameworks\n\nAccountability Mechanisms\n\nEnsuring Transparency and Explainability\n\nImplementing AI Governance Frameworks\n\nMonitoring and Continuous Improvement\n\nSecuring AI Systems\n\nAI Governance FAQs\n\n1. Understanding AI Governance\n\n2. AI Governance Challenges\n\n3. Establishing Ethical Guidelines\n\n4. Navigating Regulatory Framew

# Chunking or Splitting...

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_doc = text_splitter.split_documents(documents=doc)
splitted_doc

[Document(metadata={'source': 'https://www.paloaltonetworks.ca/cyberpedia/ai-governance'}, page_content='Table of Contents\n\nUnderstanding AI Governance\n\nAI Governance Challenges\n\nEstablishing Ethical Guidelines\n\nNavigating Regulatory Frameworks\n\nAccountability Mechanisms\n\nEnsuring Transparency and Explainability\n\nImplementing AI Governance Frameworks\n\nMonitoring and Continuous Improvement\n\nSecuring AI Systems\n\nAI Governance FAQs\n\nCloud Security\n\nWhat Is AI Governance?\n\n3 min. read\n\nTable of Contents\n\nUnderstanding AI Governance\n\nAI Governance Challenges\n\nEstablishing Ethical Guidelines\n\nNavigating Regulatory Frameworks\n\nAccountability Mechanisms\n\nEnsuring Transparency and Explainability\n\nImplementing AI Governance Frameworks\n\nMonitoring and Continuous Improvement\n\nSecuring AI Systems\n\nAI Governance FAQs\n\n1. Understanding AI Governance\n\n2. AI Governance Challenges\n\n3. Establishing Ethical Guidelines\n\n4. Navigating Regulatory Framew

# Embeddings
Vector Store - Chroma DB 

In [4]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents=splitted_doc, embedding=OpenAIEmbeddings())

C:\Users\dp971\AppData\Local\Temp\ipykernel_22660\2628869360.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = Chroma.from_documents(documents=splitted_doc, embedding=OpenAIEmbeddings())


## Integrate LLM

In [5]:
from langchain_openai import OpenAI
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    max_retries=2
)

## Design Prompt Template

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
</context>                                                                                                                                                                
Question: {input}
""")

## Chain
create stuff document chain

In [7]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm=llm,prompt=prompt)

## Retrievers

In [8]:
retriever = db.as_retriever(search_type="mmr")

### Retrieval chain(Retriever + Document chain):
    Retrieval chain takes in a user query, which is then passed to the retriever to fetch relevant documents. Those documents and original inputs are then passed to an LLM to generate a response.


In [9]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
response = retrieval_chain.invoke({"input": "What is the differnce b/w explainability and explainable AI?"})

In [11]:
response['answer']

'Answer: Explainability is the desired characteristic of an AI system, while explainable AI is the field of study and practice that aims to achieve this characteristic in AI models. In other words, explainability is the end goal, while explainable AI is the means to achieve it.'